In [ ]:
from matplotlib import pyplot
import pandas as pd
import warnings

from summer.utils import ref_times_to_dti

from autumn.core.plots.utils import REF_DATE
from autumn.core.project import get_project
from autumn.core.utils.display import pretty_print

In [ ]:
ref_times_to_dti(REF_DATE, [730.])

In [ ]:
pyplot.style.use("ggplot")
warnings.filterwarnings("ignore")

region = "wpro_australia"
project = get_project("sm_sir", region)

In [ ]:
baseline_params = project.param_set.baseline
custom_params = project.param_set.baseline#.update(
#     {
#         "time": {
#             "end": 1000.},
       
#     }
# )

In [ ]:
model = project.run_baseline_model(custom_params)
derived_df = model.get_derived_outputs_df()

model_start_time = ref_times_to_dti(REF_DATE, [custom_params["time"]["start"]])[0]
model_end_time = ref_times_to_dti(REF_DATE, [custom_params["time"]["end"]])[0]

In [ ]:
targets_dict = {
    t.data.name: pd.Series(t.data.values, index=ref_times_to_dti(model.ref_date, t.data.index)) for 
    t in project.calibration.targets
}

death_string = "accum_deaths" if region == "bangladesh" else "infection_deaths"

outputs_to_plot = ["notifications", "infection_deaths", "icu_occupancy", "hospital_occupancy"]

fig = pyplot.figure(figsize=(15, 12))
for i_out, output in enumerate(outputs_to_plot):
    axis = fig.add_subplot(2, 2, i_out + 1)
    if output in targets_dict:
        targets_dict[output].plot(ax=axis, style='.')
    if output in derived_df:
        derived_df[output].plot(ax=axis)
    axis.set_title(output.replace("_", " "))
    axis.set_xlim([model_start_time, model_end_time])

In [ ]:
derived_df[[i for i in derived_df.columns if "strain" in i]].plot.area()

In [ ]:
fig = pyplot.figure(figsize=(15, 6))
pyplot.style.use("ggplot")
axis = fig.add_subplot(1,2,1)
axis = derived_df["cdr"].plot()
axis.set_title("cdr")
axis = fig.add_subplot(1,2,2)
axis = derived_df["prop_ever_infected"].plot()
axis.set_title("prop_ever_infected")